In [31]:
import numpy
import pathlib
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

In [6]:
with open('data/5mintweets.txt') as f:
    tweetstr=f.read()

tweets=tweetstr.split('\n')
tweets[0:10]

['today is day one of the biggest project ive ever tried. it is called 17776 httpswww.sbnation.coma17776-football ',
 'congratulations to all red sox fans who are staying up past three in the morning terrified that your unbelievably successful team wont win every game. you are now an honorary fox news grandpa',
 'welcome to college football saturday',
 '.mlb 13th inning, suckers! im taking you rubes for at least four free innings of baseball. beware of jon the boring content bandit',
 'things ive done in red dead 2 so far_- fought bear_- murdered by alligator_- killed 10 lawmen after being falsely accused of murder_- called a guy a coward and he ran away_- sat by a lake, reflected_- lassoed a guy off his horse, hogtied him, put him back on his own horse, left',
 'red dead redemption 2 is a wonderful game. sending a fruit basket to the person who made it',
 'twitter offered me extended logoff time if i volunteered to show up at the mall in my twitter uniform and recruit new posters',
 '

In [21]:
# Count characters and create map to integers
chars=sorted(set(tweetstr))
# Number of distinct characters
chardict=len(chars)
charcount=len(tweetstr)
char_map={c:i for i,c in enumerate(chars)}

In [22]:
# Create sub sequences of a fixed length to feed to network
seq_length = 100
dataX = []
dataY = []
for i in range(0, charcount - seq_length):
    seq_in = tweetstr[i:i + seq_length]
    seq_out = tweetstr[i + seq_length]
    dataX.append([char_map[char] for char in seq_in])
    dataY.append(char_map[seq_out])
n_patterns = len(dataX)
print ("Total Patterns: ", n_patterns)

Total Patterns:  82090


In [28]:
# reshape X to be [samples, time steps, features]
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))
# normalize
X = X / float(chardict)
# one hot encode the output variable
y = np_utils.to_categorical(dataY)

In [30]:
# define the LSTM model
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [32]:
# define the checkpoint
pathlib.Path("out/checkpoints").mkdir(exist_ok=True)
filepath="out/checkpoints/weights-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [ ]:
# Fit model
model.fit(X, y, epochs=20, batch_size=128, callbacks=callbacks_list)

Epoch 1/20
82090/82090 [==============================] - 408s 5ms/step - loss: 3.0283

Epoch 00001: loss improved from inf to 3.02831, saving model to out/checkpoints/weights-01-3.0283.hdf5
Epoch 2/20
82090/82090 [==============================] - 403s 5ms/step - loss: 2.9023

Epoch 00002: loss improved from 3.02831 to 2.90231, saving model to out/checkpoints/weights-02-2.9023.hdf5
Epoch 3/20
82090/82090 [==============================] - 399s 5ms/step - loss: 2.8533

Epoch 00003: loss improved from 2.90231 to 2.85332, saving model to out/checkpoints/weights-03-2.8533.hdf5
Epoch 4/20
82090/82090 [==============================] - 409s 5ms/step - loss: 2.8223

Epoch 00004: loss improved from 2.85332 to 2.82226, saving model to out/checkpoints/weights-04-2.8223.hdf5
Epoch 5/20
82090/82090 [==============================] - 487s 6ms/step - loss: 2.8040

Epoch 00005: loss improved from 2.82226 to 2.80401, saving model to out/checkpoints/weights-05-2.8040.hdf5
Epoch 6/20
82090/82090 [=====